In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
import string
import numpy as np
import pickle
import textdistance

In [2]:
# Load model
from keras.models import load_model

model=load_model("./Model/Model_Chatbot_Personal_2.h5")
import json
import random

with open("./Dataset/intents.json",encoding="utf-8") as file:
    intents=json.load(file)
with open("./Dataset/words.pkl","rb") as file:
    words=pickle.load(file)
with open("./Dataset/classes.pkl","rb") as file:
    classes=pickle.load(file)
    
# print(open("./Dataset/intents.json").read())
# intents2=json.loads(open("./Dataset/intents.json",encoding="utf-8").read())

In [3]:
# print(intents)
# print(intents2)

In [4]:
print(words)
print(classes)

['hi', 'u', 'yu', 'hello', 'helo', 'hw', 'yo', 'hw', 'ar', 'anyone', 'hey', 'hola', 'good', 'day', 'hai', 'hei', 'ello', 'bye', 'see', 'later', 'goodbye', 'nice', 'chat', 'bye', 'till', 'next', 'time', 'thanks', 'thank', "'s", 'helpful', 'awesome', 'thanks', 'thanks', 'help', 'woww', 'thanks', 'abcdefghi', 'oqieo', 'wdmds', 'blablabalba', 'ckckck', 'ralwklkkww', 'pqfinwf', 'wecbwebinfg', 'dws', 'wswksowkawao', 'dd', 'idujaw', 'could', 'help', 'help', 'provide', 'helpful', 'support', 'offer', 'check', 'adverse', 'drug', 'reaction', 'open', 'adverse', 'drug', 'module', 'give', 'list', 'drug', 'cause', 'adverse', 'behavior', 'list', 'drug', 'suitable', 'patient', 'adverse', 'reaction', 'drug', 'dont', 'adverse', 'reaction', 'open', 'blood', 'pressure', 'module', 'task', 'relate', 'blood', 'pressure', 'blood', 'pressure', 'data', 'entry', 'want', 'log', 'blood', 'pressure', 'result', 'blood', 'pressure', 'data', 'management', 'want', 'search', 'blood', 'pressure', 'result', 'history', 'blo

In [5]:
lemmatizer=WordNetLemmatizer()
punc=string.punctuation
stop_words=stopwords.words("english")

# function to clean
def remove_punctuation(w_list):
    return [word for word in w_list if word not in punc]

def remove_stopwords(w_list):
    return [word for word in w_list if word not in stop_words]

def get_tag(tag):
    if tag.startswith('j'):
        return 'a'
    elif tag.startswith('v'):
        return 'v'
    elif tag.startswith('n'):
        return 'n'
    elif tag.startswith('r'):
        return 'r'
    else:
        return None

def lemmatize(w_list):
    lemmatized=[]
    tagging=pos_tag(w_list)
    for word,tag in tagging:
        tag=get_tag(tag.lower())
        if tag is None:
            lemmatized.append(word.lower())
        else:
            lemmatized.append(lemmatizer.lemmatize(word.lower(),tag))
    return lemmatized

In [12]:
def clean_up_sentence(msg):
    sentence_words=word_tokenize(msg)
    # print("Sentence word:",sentence_words)
    sentence_words=remove_punctuation(sentence_words)
    sentence_words=remove_stopwords(sentence_words)
    sentence_words=lemmatize(sentence_words)
    return sentence_words

In [13]:
def correct_typo(word):
    valid_words = set(words)
    threshold = 0.5  # Threshold for Jaccard Similarity
    similarities = []
    
    # Iterating through valid words to check for exact matches first
    for w in valid_words:
        jac_similarity = textdistance.jaccard.normalized_similarity(word, w)
        
        # If there is an exact match with Jaccard similarity 100%, return the word immediately
        if jac_similarity == 1.0:
            return w
        
        similarities.append((jac_similarity, w))
    
    # Filter words based on the threshold
    similar_words = [w for jac_similarity, w in similarities if jac_similarity >= threshold]
    
    # If there are no similar words, return the original word
    if not similar_words:
        return word
    
    # If there are similar words, return a random similar word
    return random.choice(similar_words)

In [14]:
def bag_of_words(msg,words,show_details=True):
    # tokenize
    msg=clean_up_sentence(msg)
    corrected_sentence = [correct_typo(word) for word in msg]
    # print(corrected_sentence)
    # print("Cleaned up sentence:",msg)
    
    # bag of words - matrix of N words
    bag = [0]*len(words)
    for s in corrected_sentence:
        for i,w in enumerate(words):
            if w==s:
                # assign 1 if current word is in the vocabulary (words)
                bag[i]=1
                if show_details:
                    print(f"Found in bag: {w}")
    return np.array(bag)

In [21]:
def predict_class(msg,model):
    # filter below threshold
    p=bag_of_words(msg,words,show_details=False)
    # print("p:",p)
    res=model.predict(np.array([p]),verbose=0)[0]
    # print("res:",res)
    err_threshold=0.05
    print("Result:",res.max())
    result=[[i,r] for i,r in enumerate(res) if r>err_threshold]

    if len(result)==0:
        return 0
    # print("Result:",result)
    
    # sorting
    result.sort(key=lambda x:x[1],reverse=True)
    # print("Result:",result)
    
    result_list=[]
    # for r in result:

    result_list.append({"intent":classes[result[0][0]],"probability":str(result[0][1])})
    return result_list
    

In [22]:
def get_response(int_pred,intents_json):
    tag=int_pred[0]["intent"]
    # print("Tag:",tag)
    list_of_intents=intents_json["intents"]
    # if tag in ["goodbye","thanks"]:
    #     for i in list_of_intents:
    #         if i["tag"]==tag:
    #             result=random.choice(i["responses"])
    #             break
    #     # print("Bot:",result)
    #     return result
    for i in list_of_intents:
        if i["tag"].strip()==tag.strip():
            result=random.choice(i["responses"])
            break
    return result

In [23]:
def chatbot_response(msg):
    int_predict=predict_class(msg,model)
    if int_predict == -1:
        return int_predict
    if int_predict == 0:
        return "I dont know what u mean!, please ask me something else."
    # print("Int predict:",int_predict)
    response=get_response(int_predict,intents)
    # if response==-1:
    #     return 0
    # print("Chatbot response:",response)
    return response

In [24]:
print(f'Chatbot response: {chatbot_response("what is deep learning?")}')

Sentence word: ['what', 'is', 'deep', 'learning', '?']
Cleaned up sentence: ['deep', 'learning']
p: [0 0 0 ... 0 0 0]
res: [8.11528338e-08 1.74377351e-06 4.20503099e-09 9.31660882e-09
 1.09667923e-07 3.15301278e-08 2.31587372e-09 7.02072445e-09
 2.02342483e-11 6.65393829e-09 2.92428108e-08 7.94087407e-10
 3.05557202e-09 1.94644523e-09 2.90363360e-08 7.24959870e-09
 1.96174597e-08 5.42134782e-09 1.41525556e-08 1.00179935e-08
 9.94005501e-01 4.85248641e-09 4.36076375e-09 3.14481308e-09
 6.03976558e-09 7.47070050e-09 2.40402032e-09 1.32785222e-04
 3.35327877e-09 3.70074882e-09 8.97883901e-09 3.98976496e-09
 5.43795620e-10 1.89998275e-08 8.19204349e-09 5.94482374e-09
 4.13442525e-09 8.90113494e-09 3.05173375e-09 4.15654249e-08
 2.47330734e-09 3.07444892e-09 7.20352000e-09 2.65068422e-07
 8.14685475e-09 8.07812306e-09 6.59683241e-09 6.88073021e-09
 8.88557938e-09 5.68814976e-05 5.82749404e-09 3.19908651e-07
 5.85608284e-10 3.10057516e-07 3.69159792e-09 1.93859620e-07
 3.36942230e-09 2.93452

In [30]:
# buatkan templatge pembicaraan dengan chatbotnya di sini
while True:
    msg=input("You: ")
    response=chatbot_response(msg)
    if response==-1:
        break
    print("Bot:",response.strip())
    # if response[0]=="goodbye" or response[0]=="thanks":
    #     break

Bot: Hi there, how can I help?
Bot: I can answer your questions related to machine learning or any general anser. Come on try me
Bot: Deep learning (also known as deep structured learning) is part of a broader family of machine learning methods based on artificial neural networks with representation learning.
Bot: Machine learning is a branch of computer science which deals with system programming in order to automatically learn and improve with experience. For example: Robots are programmed so that they can perform the task based on data they gather from sensors. It automatically learns programs from data.
Bot: Hello, thanks for asking
Bot: Reinforcement Learning(RL) is a type of machine learning technique that enables an agent to learn in an interactive environment by trial and error using feedback from its own actions and experiences.
Bot: Good to see you again
Bot: My pleasure


#### Flask

In [31]:
# from flask_ngrok import run_with_ngrok
from flask import Flask, jsonify, render_template, request

app=Flask(__name__)


@app.route("/")
def hello():
    return render_template('chatbot.html')

def decrypt(msg):
    # input : what+is+deep+learning
    # output : what is deep learning
    # remove + , replace with space
    
    string=msg
    new_string=string.replace("+"," ")
    
    return new_string

@app.route("/query",methods=["POST"])
def chatbot_query():
    sentence=request.form["sentence"]
    # decrypt
    dec_msg=decrypt(sentence)
    
    response=chatbot_response(dec_msg)
    
    # json_response=jsonify({"top" : {"response":response}})
    # print("Response:",response)
    return response

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/May/2024 00:20:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2024 00:20:23] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2024 00:20:23] "GET /static/Image/BotImg.png HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2024 00:20:25] "GET /static/Image/UserImg.jpg HTTP/1.1" 304 -
127.0.0.1 - - [31/May/2024 00:20:25] "POST /query HTTP/1.1" 200 -


['hi']
Result: 0.9999999


127.0.0.1 - - [31/May/2024 00:20:29] "POST /query HTTP/1.1" 200 -


['hei']
Result: 0.99999976


...

['machine', 'leraning']
Result: 0.99916697


127.0.0.1 - - [31/May/2024 00:20:52] "POST /query HTTP/1.1" 200 -


['deep', 'leraning']
Result: 0.9632643


127.0.0.1 - - [31/May/2024 00:21:03] "POST /query HTTP/1.1" 200 -


['woww', 'thanks', 'awesome']
Result: 1.0


In [ ]:
!pip list

Package                   Version
------------------------- ----------
anyio                     4.1.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.1
beautifulsoup4            4.12.2
bleach                    6.1.0
blinker                   1.7.0
certifi                   2021.5.30
cffi                      1.16.0
chardet                   4.0.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.2.0
contourpy                 1.2.0
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
distlib                   0.3.7
executing                 2.0.1
fastjsonschema            2.19.0
filelock                  3.13.1
Flask                     3.0.2
flask-ngrok         


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\micha\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip
